In [1]:
import numpy as np
import sys
import pathlib
path=pathlib.Path("/home/quim/stg/jungsu/robotics-course/")
sys.path.append(str(path / "build"))
import libry as ry
import time

**ry-c++-log** ry.cpp:init_LogToPythonConsole:34(0) initializing ry log callback

**ry-c++-log** util.cpp:initCmdLine:545(1) ** cmd line arguments: 'rai-pybind -python '

** INFO:ry.cpp:init_LogToPythonConsole:34(0) initializing ry log callback
**ry-c++-log** util.cpp:initCmdLine:549(1) ** run path: '/home/quim/stg/jungsu/roboticslab/code/e2'** INFO:util.cpp:initCmdLine:545(1) ** cmd line arguments: 'rai-pybind -python '


**ry-c++-log** graph.cpp:initParameters:1379(1) ** parsed parameters:
{python}

** INFO:util.cpp:initCmdLine:549(1) ** run path: '/home/quim/stg/jungsu/roboticslab/code/e2'

** INFO:graph.cpp:initParameters:1379(1) ** parsed parameters:
{python}



In [4]:
# Here we do not need a simulation world
# adding a configuration world
C = ry.Config()
C.addFile(str(path  / "scenarios/pandasTable.g"))
D = C.view()

In [5]:
obj = C.addFrame("object")
obj.setPosition([.2,.2,1.5])
obj.setQuaternion([1,0,1,0])
obj.setShape(ry.ST.capsule, [.2,.02])
obj.setColor([1,0,1])

In [6]:
IK = C.komo_IK(False)
mode = 1 # 0 : positionRel = 0 0 0 ; 1 PositionRel constraints
# of ex2-3 
l = .2 # length is .2
if mode == 0:
    IK.addObjective([1.], 
                    ry.FS.positionRel, 
                    ["object", "R_gripperCenter"], 
                    ry.OT.eq, 
                    [1e2],
                    [0.,0.,-.0]);
if mode == 1:

    IK.addObjective([1.], 
                    ry.FS.positionRel, 
                    ["R_gripperCenter","object"], 
                    ry.OT.eq, 
                    1e2*np.array([[1,0,0],[0,1,0]]),
                    [0.,0.,-.0]);
# NOTE: we can also use:
#     IK.addObjective([1.], 
#                     ry.FS.positionRel, 
#                     ["R_gripperCenter","object"], 
#                     ry.OT.eq, 
#                     [1e2, 0, 0],
#                     [0.,0.,-.0]);
    
#     IK.addObjective([1.], 
#                     ry.FS.positionRel, 
#                     ["R_gripperCenter","object"], 
#                     ry.OT.eq, 
#                     [0, 1e2, 0],
#                     [0.,0.,-.0]);
    
    IK.addObjective([1.], 
                    ry.FS.positionRel, 
                    ["R_gripperCenter","object"], 
                    ry.OT.ineq, 
                    [0, 0, 1e2],
                    [0.,0.,l/2]);
    IK.addObjective([1.], 
                    ry.FS.positionRel, 
                    ["R_gripperCenter","object"], 
                    ry.OT.ineq, 
                    [0, 0, -1e2],
                    [0.,0.,-l/2]);
    
    # we can add cost, to grasp in the middle if possible
    # IK.addObjective([1.], 
    #             ry.FS.positionRel, 
    #             ["R_gripperCenter","object"], 
    #             ry.OT.sos, 
    #             [0, 0, 1e1]);





In [7]:
# Calling the optimizer (True means random initialization/restart)
IK.optimize()
IK.getReport()

** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:220
** optimization time:0.000777 (kin:2.2e-05 coll:0 feat:0.000129 newton: -1.91807) setJointStateCount:5
   sos:0.00515725 ineq:0 eq:0.0218516


{'F_qItself/1-#28': {'order': 1.0, 'type': 'sos', 'sos': 0.005157246827317101},
 'F_qQuaternionNorms/0-#110': {'order': 0.0, 'type': 'eq'},
 'F_PositionRel/0-R_gripperCenter-object': {'order': 0.0, 'type': 'ineq'},
 'sos': 0.005157246827317101,
 'ineq': 0.0,
 'eq': 0.02185157705097751,
 'f': 0.0}

In [8]:
C.setFrameState( IK.getFrameState(0) )